In [8]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='ae1a755d-e162-4f07-9f5a-130d2280e78e', project_access_token='p-aa90b9b21de435c3f4c94494a24b5c5e69d030f8')
pc = project.project_context


# Extract the Personal Identifiable Information (PII) using Watson NLP

<h2>Use Case</h2>

This notebook demonstrates how to extract PII entities using Watson NLP Pertained models also demonstrates how to prepare custom train models. PII (Personally extraction is the process of identifying and extracting personal information from a document or dataset. This information can include names, addresses, phone numbers, email addresses, Social Security numbers, Credit Card number, and other types of information that can be used to identify an individual. 


<h2>What you'll learn in this notebook</h2>

Watson NLP offers Pertained Models for various NLP tasks also provides fine-tune functionality for custom training. This notebooks shows:

* <b>RBR</b>:  A Rule-Based Reasoner (RBR) in NLP works by using a set of predefined rules to process and understand natural language input. These rules are used to identify specific patterns or structures in the input text and determine the meaning of the text based on those patterns.


* <b>BILSTM</b>: the BiLSTM network would take the preprocessed text as input and learn to identify patterns and relationships between words that are indicative of PII data. The BiLSTM network would then output a probability score for each word in the text, indicating the likelihood that the word is part of a PII entity. The BiLSTM network may also be trained to recognize specific entities such as names, addresses, phone numbers, email addresses, etc.


* <b>SIRE</b>: Statistical Information and Relation Extraction (SIRE) is a technique used in natural language processing (NLP) to extract specific information and relationships from text. It involves using machine learning algorithms to identify and extract structured data such as entities, attributes, and relations from unstructured text. SIRE is used in a variety of applications, including information extraction, knowledge graph construction, and question answering. SIRE typically uses supervised learning approach, where a model is trained using annotated examples of text and the corresponding structured data. The model can then be used to extract the same information from new, unseen text.


* <b>Bert</b>: IBM Watson NLP BERT uses a pre-trained version of BERT that was trained on a large corpus of text data. The pre-trained model can be fine-tuned on a specific task such as text classification, named entity recognition, and more. The BERT architecture consists of an encoder network that is made up of multiple layers of transformer blocks. Each transformer block includes a self-attention mechanism and a feed-forward neural network.


* <b>Transformer</b>: This model is a neural network architecture that is used in natural language processing tasks such as language translation, text summarization, and language generation. It is based on self-attention mechanism and can be used to extract information such as named entities, relationships and sentiments from the text.

## Table of Contents


1.  [Before you start](#beforeYouStart)
1.	[Load Entity PII Models](#LoadModel)
1.  [Load PII XLSX Dataset from Data Assets](#Loaddata)
1.  [TrainingData](#TrainingData)
1.  [Watson NLP Models](#NLPModels)    
    1. [BiLSTM Fine-tuned](#BILSTMFINE)
    1. [SIRE Fine-tuned](#SIRETune)
    1. [Transformer Fine-tuned](#TransTUne)
    
1.  [Testing With Hanzo's Test Dataset](#Testing)    
1.  [Summary](#summary)

<a id="beforeYouStart"></a>
### 1. Before you start


<div class="alert alert-block alert-danger">
<b>Stop kernel of other notebooks.</b></div>

**Note:** If you have other notebooks currently running with the _Default Python 3.8 + Watson NLP XS_ environment, **stop their kernels** before running this notebook. All these notebooks share the same runtime environment, and if they are running in parallel, you may encounter memory issues. To stop the kernel of another notebook, open that notebook, and select _File > Stop Kernel_.

<div class="alert alert-block alert-warning">
<b>Set Project token.</b></div>

Before you can begin working on this notebook in Watson Studio in Cloud Pak for Data as a Service, you need to ensure that the project token is set so that you can access the project assets via the notebook.

When this notebook is added to the project, a project access token should be inserted at the top of the notebook in a code cell. If you do not see the cell above, add the token to the notebook by clicking **More > Insert project token** from the notebook action bar.  By running the inserted hidden code cell, a project object is created that you can use to access project resources.

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)

<div class="alert alert-block alert-info">
<b>Tip:</b> Cell execution</div>

Note that you can step through the notebook execution cell by cell, by selecting Shift-Enter. Or you can execute the entire notebook by selecting **Cell -> Run All** from the menu.

In [9]:
import json
import pandas as pd
import watson_nlp
from watson_nlp import data_model as dm
from watson_nlp.toolkit.entity_mentions_utils import prepare_train_from_json

In [10]:
# Silence Tensorflow warnings
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

<a id="LoadModel"></a>
### 2. Load Entity PII Models

In [11]:
# Load a syntax model to split the text into sentences and tokens
syntax_model = watson_nlp.load(watson_nlp.download('syntax_izumo_en_stock'))
# Load bilstm model in WatsonNLP
bilstm_model = watson_nlp.load(watson_nlp.download('entity-mentions_bilstm_en_pii'))
# Load rbr model in WatsonNLP
rbr_model = watson_nlp.load(watson_nlp.download('entity-mentions_rbr_multi_pii'))
# Download the GloVe model to be used as embeddings in the BiLSTM
glove_model = watson_nlp.load(watson_nlp.download('embedding_glove_en_stock'))
# Download the algorithm template
mentions_train_template = watson_nlp.load(watson_nlp.download('file_path_entity-mentions_sire_multi_template-crf'))
# Download the feature extractor
default_feature_extractor = watson_nlp.load(watson_nlp.download('feature-extractor_rbr_entity-mentions_sire_en_stock'))
# Download and load the pretrained model resource
#pretrained_model_resource = watson_nlp.load(watson_nlp.download('pretrained-model_roberta-base_v2-8-0_llm_transformer_lang_en_cased_2022-05-06-052653'))

<a id="Loaddata"></a>
### 3. Load PII XLSX Dataset from Data Assets

In [12]:
import os, types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='o0avUc3SDky2d6pNzjuewCSTPPX7tQNz6BKKvL37nBL3',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'watsoncore-donotdelete-pr-olkxvfa8bk0pb1'
object_key = '10-MB-Test.xlsx'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']

df = pd.read_excel(body.read())
df = df.dropna()
df.head()

,First and Last Name,SSN,Credit Card Number,First and Last Name.1,SSN.1,Credit Card Number.1,First and Last Name.2,SSN.2,Credit Card Number.2,First and Last Name.3,...,Credit Card Number.3,First and Last Name.4,SSN.4,Credit Card Number.4,First and Last Name.5,SSN.5,Credit Card Number.5,First and Last Name.6,SSN.6,Credit Card Number.6
1,Robert Aragon,489-36-8350,4929-3813-3266-4295,Robert Aragon,489-36-8351,4929-3813-3266-4296,Robert Aragon,489-36-8352,4929-3813-3266-4297,Robert Aragon,...,4929-3813-3266-4298,Robert Aragon,489-36-8354,4929-3813-3266-4299,Robert Aragon,489-36-8355,4929-3813-3266-4300,Robert Aragon,489-36-8355,4929-3813-3266-4300
2,Ashley Borden,514-14-8905,5370-4638-8881-3020,Ashley Borden,514-14-8906,5370-4638-8881-3021,Ashley Borden,514-14-8907,5370-4638-8881-3022,Ashley Borden,...,5370-4638-8881-3023,Ashley Borden,514-14-8909,5370-4638-8881-3024,Ashley Borden,514-14-8910,5370-4638-8881-3025,Ashley Borden,514-14-8910,5370-4638-8881-3025
3,Thomas Conley,690-05-5315,4916-4811-5814-8111,Thomas Conley,690-05-5316,4916-4811-5814-8112,Thomas Conley,690-05-5317,4916-4811-5814-8113,Thomas Conley,...,4916-4811-5814-8114,Thomas Conley,690-05-5319,4916-4811-5814-8115,Thomas Conley,690-05-5320,4916-4811-5814-8116,Thomas Conley,690-05-5320,4916-4811-5814-8116
4,Susan Davis,421-37-1396,4916-4034-9269-8783,Susan Davis,421-37-1397,4916-4034-9269-8784,Susan Davis,421-37-1398,4916-4034-9269-8785,Susan Davis,...,4916-4034-9269-8786,Susan Davis,421-37-1400,4916-4034-9269-8787,Susan Davis,421-37-1401,4916-4034-9269-8788,Susan Davis,421-37-1401,4916-4034-9269-8788
5,Christopher Diaz,458-02-6124,5299-1561-5689-1938,Christopher Diaz,458-02-6125,5299-1561-5689-1939,Christopher Diaz,458-02-6126,5299-1561-5689-1940,Christopher Diaz,...,5299-1561-5689-1941,Christopher Diaz,458-02-6128,5299-1561-5689-1942,Christopher Diaz,458-02-6129,5299-1561-5689-1943,Christopher Diaz,458-02-6129,5299-1561-5689-1943


<a id="TrainingData"></a>
### 4. Preparing Training Data

Let's generate sentences using the columns of PII information. Ideally, the sentences would include name, SSN, and credit card number in context.

In [13]:
def format_data(df, name_col, ssn_col, ccn_col):  
    import random
    
    train_list = []
    for i in range(1, len(df)):
        name = df[name_col][i] 
        ssn = str(df[ssn_col][i])
        ccn = str(df[ccn_col][i])
        
        text1 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (name, ssn, ccn)
        text2 = "%s is my social security number. The name on my American Express card %s is %s." % (ssn, ccn, name)
        text3 = ""
        text = random.choice([text1, text2])

        name_begin = text.find(name)
        name_end = text.find(name) + len(name)
        ssn_begin = text.find(ssn)
        ssn_end = text.find(ssn) + len(ssn)
        ccn_begin = text.find(ccn)
        ccn_end = text.find(ccn) + len(ccn)

        data = {
                    "text": text,
                    "mentions": [
                        {
                            "location": {
                                "begin": name_begin,
                                "end": name_end
                            },
                            "text": name,
                            "type": "Name"
                        },
                        {
                            "location": {
                                "begin": ssn_begin,
                                "end": ssn_end
                            },
                            "text": ssn,
                            "type": "SocialSecurityNumber"
                        },
                        {
                            "location": {
                                "begin": ccn_begin,
                                "end": ccn_end
                            },
                            "text": ccn,
                            "type": "CreditCardNumber"
                        }
                    ]   
                }

        train_list.append(data)
    return train_list

In [14]:
train_list = format_data(df=df, name_col='First and Last Name', ssn_col='SSN', ccn_col='Credit Card Number')

Save the sentences into a json training file and a json dev file. This will save the file to the runtime local as well as the project data assets.

In [15]:
with open('PII_text_train.json', 'w') as f:
    json.dump(train_list, f)
project.save_data('PII_text_train.json', data=json.dumps(train_list), overwrite=True)

{'file_name': 'PII_text_train.json',
 'message': 'File saved to project storage.',
 'bucket_name': 'watsoncore-donotdelete-pr-olkxvfa8bk0pb1',
 'asset_id': '216b85be-aabe-4ff6-b264-acd101222fbc'}

In [16]:
dev_list = format_data(df=df, name_col='First and Last Name.1', ssn_col='SSN.1', ccn_col='Credit Card Number.1')

In [17]:
with open('PII_text_dev.json', 'w') as f:
    json.dump(dev_list, f)
project.save_data('PII_text_dev.json', data=json.dumps(dev_list), overwrite=True)

{'file_name': 'PII_text_dev.json',
 'message': 'File saved to project storage.',
 'bucket_name': 'watsoncore-donotdelete-pr-olkxvfa8bk0pb1',
 'asset_id': '76834e31-ab93-4aca-b86b-ce6e71476478'}

In [18]:
text = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][1], df['Credit Card Number'][1])

In [19]:
train_data = dm.DataStream.from_json_array("PII_text_train.json")
train_iob_stream = prepare_train_from_json(train_data, syntax_model)
dev_data = dm.DataStream.from_json_array("PII_text_dev.json")
dev_iob_stream = prepare_train_from_json(dev_data, syntax_model)

<a id="NLPModels"></a>
### 5. Watson NLP Models

<a id="BILSTMFINE"></a>

### BiLSTM Fine-tuned

In [ ]:
bilstm_custom = bilstm_model.train(train_iob_stream, 
                                   dev_iob_stream, 
                                   embedding=glove_model.embedding,
                                   #vocab_tags=None, 
                                   #char_embed_dim=32, 
                                   #dropout=0.2, 
                                   #num_oov_buckets=1, 
                                   num_train_epochs=5,
                                   num_conf_epochs=5, 
                                   checkpoint_interval=5, 
                                   learning_rate=0.005, 
                                   #shuffle_buffer=2000, 
                                   #char_lstm_size=64, 
                                   #char_bidir=False, 
                                   lstm_size=16, 
                                   #train_batch_size=32, 
                                   #lower_case=False, 
                                   #embedding_lowercase=True, 
                                   #keep_model_artifacts=False)
                                  )

2066/2138 [===========================>..] - ETA: 6s - loss: 1.7969e-04

In [ ]:
project.save_data('bilstm_pii_custom', data=bilstm_custom.as_file_like_object(), overwrite=True)

In [ ]:
syntax_result = syntax_model.run(text)
bilstm_result = bilstm_custom.run(syntax_result)
bilstm_result

<a id="SIRETune"></a>

### SIRE Fine-tuned


In [ ]:
help(watson_nlp.blocks.entity_mentions.SIRE)

In [ ]:
sire_custom = watson_nlp.blocks.entity_mentions.SIRE.train(train_iob_stream, 
                                                           'en', 
                                                           mentions_train_template,
                                                           feature_extractors=[default_feature_extractor])

In [ ]:
project.save_data('sire_pii_custom', data=sire_custom.as_file_like_object(), overwrite=True)

In [ ]:
syntax_result = syntax_model.run(text)
sire_result = sire_custom.run(syntax_result)
sire_result

<a id="TransTUne"></a>
### Transformer Fine-tuned


In [ ]:
#help(watson_nlp.blocks.entity_mentions.Transformer)

In [ ]:
# Download and load the pretrained model resource
pretrained_model_resource = watson_nlp.load(watson_nlp.download('pretrained-model_watbert_multi_transformer_multi_uncased'))

In [ ]:
'''
transformer_custom = watson_nlp.blocks.entity_mentions.Transformer.train(train_iob_stream,
                                                                         dev_iob_stream,
                                                                         pretrained_model_resource,
                                                                         lr=0.005,
                                                                         num_train_epochs=5,
                                                                         #per_device_train_batch_size=32,
                                                                         #per_device_eval_batch_size=32,
                                                                         max_seq_length=205,
                                                                         seed=1,
                                                                         keep_model_artifacts=True)
'''
transformer_custom = watson_nlp.blocks.entity_mentions.Transformer.train(train_iob_stream,
                                                                         dev_iob_stream,
                                                                         pretrained_model_resource,
                                                                         num_train_epochs=8,
                                                                         learning_rate=3e-5,
                                                                         per_device_train_batch_size=1,
                                                                         per_device_eval_batch_size=32)

In [ ]:
project.save_data('transformer_pii_custom', data=transformer_custom.as_file_like_object(), overwrite=True)

In [ ]:
#Test the custom train transformer model 
syntax_result = syntax_model.run(test1)
transformer_result = transformer_custom.run(syntax_result)
transformer_result

<a id="summary"></a>
## 6. Summary

<span style="color:blue">This notebook shows you how to use the Watson NLP library to:
1. Extract PII Using Custom or Fine tune Models </span>

Please note that this content is made available by IBM Build Lab to foster Embedded AI technology adoption. The content may include systems & methods pending patent with USPTO and protected under US Patent Laws. For redistribution of this content, IBM will use release process. For any questions please log an issue in the GitHub.

Developed by IBM Build Lab

Copyright - 2022 IBM Corporation